In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import glob
import os 
import numpy as np # linear algebra
import pandas as pan # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plotter
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.optimizers import Adam
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h2> Brains. <h2/>

In [ ]:
#walk the images
MASTER = "//kaggle//input//brain-mri-images-for-brain-tumor-detection"
images = [i for i in glob.glob(MASTER +"//*//*")]#trying to be very incomprehensible here
np.random.shuffle(images)
image_labels = [os.path.dirname(i).split("/")[-1] for i in images]

In [ ]:
#zip into df for visualization and stuff
image_with_labels = zip(images, image_labels)
cols = ["Images", "Labels"]
data_frame = pan.DataFrame(image_with_labels, columns = cols)

In [ ]:
sns.countplot(x = data_frame["Labels"])

In [ ]:
data_frame = data_frame[data_frame["Labels"] != "brain_tumor_dataset"]
sns.countplot(x = data_frame["Labels"])

In [ ]:
#lets see some stuff
test_arr = plotter.imread(data_frame.iloc[19]["Images"])
plotter.imshow(test_arr)

In [ ]:
#seems the image sizes are rather large in general, resizing to around 300, 300 should do the trick
IMG_SIZE = (300, 300, 1)#it's black and white, right???
#again, anyone is welcome to fork and visualize
#encode the label
mapper = {label: num for num, label in enumerate(data_frame["Labels"].unique())}
#train-test-split next
features = data_frame["Images"]
label = data_frame["Labels"].map(mapper)
feature_train, feature_test, label_train, label_test = train_test_split(features, label, test_size = 0.2, random_state = 42)

#define preprocessing pipeline
def preprocessor(image, label):
    apply_effect = tf.random.uniform([]) < 0.2
    image = tf.io.read_file(image)
    image = tf.image.decode_jpeg(image, channels = 1)
    image = tf.image.resize(image, IMG_SIZE[: 2])
    image = tf.image.stateless_random_brightness(image, max_delta = 0.2, seed = (42, 101))
    image = tf.cond(apply_effect, lambda : tf.image.stateless_random_flip_up_down(image, seed = (42, 101)), lambda: image)
    image = tf.cond(apply_effect, lambda: tf.image.stateless_random_flip_left_right(image, seed = (42, 101)), lambda: image)
    image = image/255
    return image, tf.one_hot(label, depth = 1) #[0, 1], [1, 0], right?

#create dataset objects from np arrays, zip, then apply transformation
feature_train_dataset = tf.data.Dataset.from_tensor_slices(feature_train)
feature_test_dataset = tf.data.Dataset.from_tensor_slices(feature_test)
label_train_dataset = tf.data.Dataset.from_tensor_slices(label_train)
label_test_dataset = tf.data.Dataset.from_tensor_slices(label_test)

train = tf.data.Dataset.zip((feature_train_dataset, label_train_dataset))
test = tf.data.Dataset.zip((feature_test_dataset, label_test_dataset))

train = train.map(preprocessor).batch(64).shuffle(128).prefetch(128)
test = test.map(preprocessor).batch(128).prefetch(128)

In [ ]:
#build model
image_learner = Sequential()
image_learner.add(Conv2D(16, (2, 2), activation = "relu", input_shape = IMG_SIZE))
image_learner.add(MaxPool2D((2, 2)))
image_learner.add(Conv2D(32, (2, 2), activation = "relu"))
image_learner.add(MaxPool2D((2, 2)))
image_learner.add(Conv2D(64, (2, 2), activation = "relu"))
image_learner.add(MaxPool2D((2, 2)))
image_learner.add(Flatten())
image_learner.add(Dense(units = 128, activation = "relu"))
image_learner.add(Dense(units = 1, activation = "sigmoid"))

optimizer = Adam(learning_rate = 1e-3)
image_learner.compile(loss = "binary_crossentropy", metrics = ["accuracy"], optimizer = optimizer)
image_learner.summary()

In [ ]:
es = EarlyStopping(patience = 3)
image_learner.fit(train, validation_data = test, callbacks = [es], epochs = 25)

In [ ]:
image_learner.evaluate(test)